<a href="https://colab.research.google.com/github/kwabena369/101_testing/blob/main/ag_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# AG NEWS CLASSIFICATION PROJECT
# Data Prep + Training Combined

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,  # ← Fixed: Capital A
    Trainer,
    TrainingArguments
)
import torch

print("="*50)
print("AG NEWS 4-CLASS CLASSIFICATION")
print("="*50)

# ===== PART 1: DATA PREPARATION =====
print("\n📊 PART 1: DATA PREPARATION")

# Load dataset
print("\n1. Loading AG News dataset...")
dataset = load_dataset("ag_news")
df = pd.DataFrame(dataset['train'])

# Sample for quick experiments
df = df.sample(10000, random_state=42)
df = df.dropna()
print(f"✓ Using {len(df)} samples")

# Text analysis
df['text_count'] = df['text'].str.len()
df['word_count'] = df['text'].str.split().str.len()

print("\n2. Text statistics:")
print(df['word_count'].describe())

# Visualizations
print("\n3. Creating visualizations...")
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
sns.countplot(x='label', data=df)
plt.title('Label Distribution')
plt.xlabel('Class (0=World, 1=Sports, 2=Business, 3=Sci/Tech)')

plt.subplot(1, 2, 2)
sns.histplot(df['word_count'], bins=30, kde=True)
plt.title('Word Count Distribution')

plt.tight_layout()
plt.savefig('ag_news_analysis.png')
print("✓ Saved visualization: ag_news_analysis.png")

# Label distribution
print("\n4. Label distribution:")
print(df['label'].value_counts().sort_index())

# Rename column for Trainer
df = df.rename(columns={"label": "labels"})

# Split data
print("\n5. Splitting data...")
train_df, temp = train_test_split(df, test_size=0.3, random_state=42, stratify=df['labels'])
test_df, val_df = train_test_split(temp, test_size=0.5, random_state=42, stratify=temp['labels'])

print(f"Train: {len(train_df)} samples")
print(f"Val: {len(val_df)} samples")
print(f"Test: {len(test_df)} samples")

# Save CSVs
train_df.to_csv('ag_news_train.csv', index=False)
test_df.to_csv('ag_news_test.csv', index=False)
val_df.to_csv('ag_news_val.csv', index=False)
print("✓ Saved CSV files")

# ===== PART 2: MODEL TRAINING =====
print("\n\n🤖 PART 2: MODEL TRAINING")

# Load CSVs and convert to Dataset
print("\n1. Loading data...")
train_dataset = Dataset.from_pandas(train_df[['text', 'labels']])
val_dataset = Dataset.from_pandas(val_df[['text', 'labels']])
test_dataset = Dataset.from_pandas(test_df[['text', 'labels']])

# Load tokenizer
print("\n2. Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenization function
def tokenize_function(text_content):
    return tokenizer(
        text_content['text'],
        padding='max_length',
        truncation=True,
        max_length=128  # Perfect for AG News!
    )

# Apply tokenization
print("\n3. Tokenizing data...")
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
print("✓ Tokenization complete")

# Compute metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Use 'macro' for balanced multi-class data
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='macro', zero_division=0
    )
    acc = accuracy_score(labels, preds)

    print(f"\n📊 Metrics: Acc={acc:.3f}, F1={f1:.3f}, Prec={precision:.3f}, Rec={recall:.3f}")

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Load model
print("\n4. Loading model...")
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=4  # 4 classes!
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"✓ Model loaded on {device}")

# Training arguments
training_args = TrainingArguments(
    output_dir='./results_agnews',
    num_train_epochs=1,
    eval_strategy="steps",  # ← Fixed typo
    eval_steps=100,
    save_steps=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    logging_steps=50,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # ← Fixed typo
    save_total_limit=2
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # ← Fixed: added comma
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train!
print("\n5. Training model...")
print("This should take ~5-10 minutes\n")
trainer.train()

print("\n✓ Training complete!")

# Evaluate on test set
print("\n6. Testing on unseen data...")
test_results = trainer.evaluate(test_dataset)

print("\n" + "="*50)
print("FINAL TEST RESULTS:")
print("="*50)
print(f"Accuracy:  {test_results['eval_accuracy']:.3f} ({test_results['eval_accuracy']*100:.1f}%)")
print(f"F1 Score:  {test_results['eval_f1']:.3f}")
print(f"Precision: {test_results['eval_precision']:.3f}")
print(f"Recall:    {test_results['eval_recall']:.3f}")

# Save model
print("\n7. Saving model...")
model.save_pretrained('./ag_news_classifier')
tokenizer.save_pretrained('./ag_news_classifier')

# Save model info
import json
model_info = {
    'model_name': 'distilbert-base-uncased',
    'num_labels': 4,
    'max_length': 128,
    'test_accuracy': test_results['eval_accuracy'],
    'test_f1': test_results['eval_f1'],
    'label_map': {
        0: 'World',
        1: 'Sports',
        2: 'Business',
        3: 'Sci/Tech'
    }
}

with open('./ag_news_classifier/model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print("✓ Model saved!")

# ===== TEST WITH EXAMPLES =====
print("\n" + "="*50)
print("TESTING WITH REAL EXAMPLES")
print("="*50)

def predict_category(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = outputs.logits.argmax(-1).item()
        confidence = torch.softmax(outputs.logits, dim=-1).max().item()

    categories = ['World', 'Sports', 'Business', 'Sci/Tech']
    return categories[prediction], confidence

# Test examples
test_examples = [
    "Wall Street stocks rally on strong earnings",
    "Scientists discover new exoplanet",
    "Manchester United wins Premier League",
    "President announces new policy on climate change"
]

print("\nPredictions:")
for i, text in enumerate(test_examples, 1):
    category, confidence = predict_category(text)
    print(f"\n{i}. '{text}'")
    print(f"   → {category} ({confidence*100:.1f}% confident)")

print("\n" + "="*50)
print("✅ PROJECT COMPLETE!")
print("="*50)
print("\nYou just built a 4-class news classifier from scratch!")
print("Your skills are leveling up! 🚀")

AG NEWS 4-CLASS CLASSIFICATION

📊 PART 1: DATA PREPARATION

1. Loading AG News dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

...

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.487200,0.425323,0.860667,0.856246,0.880344,0.859005
200,0.359600,0.307717,0.896667,0.896166,0.897357,0.896186



📊 Metrics: Acc=0.861, F1=0.856, Prec=0.880, Rec=0.859


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



📊 Metrics: Acc=0.897, F1=0.896, Prec=0.897, Rec=0.896


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
